In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os


def random_init(size):
    in_dim = size[0]
    stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=stddev)

In [9]:
X = tf.placeholder(tf.float32, shape=[None, 784])

D_W1 = tf.Variable(xavier_init([784, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]


Z = tf.placeholder(tf.float32, shape=[None, 100])

G_W1 = tf.Variable(xavier_init([100, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(xavier_init([128, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [4]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [5]:
def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob

In [6]:
def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit

In [7]:
# plotting function

def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [10]:
G_sample = generator(Z)
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
G_loss = -tf.reduce_mean(tf.log(D_fake))

In [12]:
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

mb_size = 128
Z_dim = 100

mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [20]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')

i = 0

for it in range(100001):
    if it % 1000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})

        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(mb_size)

    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(mb_size, Z_dim)})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()


Iter: 0
D loss: 1.395
G_loss: 2.363

Iter: 1000
D loss: 0.01755
G_loss: 6.23

Iter: 2000
D loss: 0.07417
G_loss: 4.664

Iter: 3000
D loss: 0.1373
G_loss: 5.251

Iter: 4000
D loss: 0.2455
G_loss: 4.528

Iter: 5000
D loss: 0.3596
G_loss: 4.776

Iter: 6000
D loss: 0.4517
G_loss: 3.387

Iter: 7000
D loss: 0.4119
G_loss: 3.394

Iter: 8000
D loss: 0.5071
G_loss: 3.155

Iter: 9000
D loss: 0.49
G_loss: 2.911

Iter: 10000
D loss: 0.6163
G_loss: 2.672

Iter: 11000
D loss: 0.6626
G_loss: 2.255

Iter: 12000
D loss: 0.6176
G_loss: 2.602

Iter: 13000
D loss: 0.7386
G_loss: 2.321

Iter: 14000
D loss: 0.7507
G_loss: 2.297

Iter: 15000
D loss: 0.8406
G_loss: 2.22

Iter: 16000
D loss: 0.8676
G_loss: 2.486

Iter: 17000
D loss: 0.7375
G_loss: 2.002

Iter: 18000
D loss: 0.7664
G_loss: 2.027

Iter: 19000
D loss: 0.5779
G_loss: 2.401

Iter: 20000
D loss: 0.6853
G_loss: 2.115

Iter: 21000
D loss: 0.7431
G_loss: 2.338

Iter: 22000
D loss: 0.8412
G_loss: 2.267

Iter: 23000
D loss: 0.6251
G_loss: 2.185

Iter: 24

/Users/chanes/miniconda3/lib/python3.5/site-packages/matplotlib/colors.py:941: UserWarning: Warning: converting a masked element to nan.
  vmin = float(vmin)
/Users/chanes/miniconda3/lib/python3.5/site-packages/matplotlib/colors.py:942: UserWarning: Warning: converting a masked element to nan.
  vmax = float(vmax)


Iter: 85000
D loss: nan
G_loss: nan

Iter: 86000
D loss: nan
G_loss: nan

Iter: 87000
D loss: nan
G_loss: nan

Iter: 88000
D loss: nan
G_loss: nan



KeyboardInterrupt: 